# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f28cc7d3b80>
├── 'a' --> tensor([[ 0.9911, -0.1099, -2.0049],
│                   [-1.0945,  0.2374, -0.2804]])
└── 'x' --> <FastTreeValue 0x7f28cc7d3a60>
    └── 'c' --> tensor([[ 0.3893, -2.6018,  0.6255, -1.8468],
                        [-0.0036, -3.4258,  0.8249,  0.3328],
                        [ 2.0975, -2.0692, -0.6502, -1.6187]])

In [4]:
t.a

tensor([[ 0.9911, -0.1099, -2.0049],
        [-1.0945,  0.2374, -0.2804]])

In [5]:
%timeit t.a

65.9 ns ± 0.0135 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f28cc7d3b80>
├── 'a' --> tensor([[-0.1365, -0.7505,  0.4394],
│                   [-0.8801, -0.4917, -0.4603]])
└── 'x' --> <FastTreeValue 0x7f28cc7d3a60>
    └── 'c' --> tensor([[ 0.3893, -2.6018,  0.6255, -1.8468],
                        [-0.0036, -3.4258,  0.8249,  0.3328],
                        [ 2.0975, -2.0692, -0.6502, -1.6187]])

In [7]:
%timeit t.a = new_value

66 ns ± 0.199 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9911, -0.1099, -2.0049],
               [-1.0945,  0.2374, -0.2804]]),
    x: Batch(
           c: tensor([[ 0.3893, -2.6018,  0.6255, -1.8468],
                      [-0.0036, -3.4258,  0.8249,  0.3328],
                      [ 2.0975, -2.0692, -0.6502, -1.6187]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9911, -0.1099, -2.0049],
        [-1.0945,  0.2374, -0.2804]])

In [11]:
%timeit b.a

57.9 ns ± 0.167 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[0.0410, 0.9780, 0.6365],
               [2.6338, 0.9338, 0.4966]]),
    x: Batch(
           c: tensor([[ 0.3893, -2.6018,  0.6255, -1.8468],
                      [-0.0036, -3.4258,  0.8249,  0.3328],
                      [ 2.0975, -2.0692, -0.6502, -1.6187]]),
       ),
)

In [13]:
%timeit b.a = new_value

523 ns ± 2.35 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

828 ns ± 0.693 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.6 µs ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 719 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 511 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f27f7582a90>
├── 'a' --> tensor([[[ 0.9911, -0.1099, -2.0049],
│                    [-1.0945,  0.2374, -0.2804]],
│           
│                   [[ 0.9911, -0.1099, -2.0049],
│                    [-1.0945,  0.2374, -0.2804]],
│           
│                   [[ 0.9911, -0.1099, -2.0049],
│                    [-1.0945,  0.2374, -0.2804]],
│           
│                   [[ 0.9911, -0.1099, -2.0049],
│                    [-1.0945,  0.2374, -0.2804]],
│           
│                   [[ 0.9911, -0.1099, -2.0049],
│                    [-1.0945,  0.2374, -0.2804]],
│           
│                   [[ 0.9911, -0.1099, -2.0049],
│                    [-1.0945,  0.2374, -0.2804]],
│           
│                   [[ 0.9911, -0.1099, -2.0049],
│                    [-1.0945,  0.2374, -0.2804]],
│           
│                   [[ 0.9911, -0.1099, -2.0049],
│                    [-1.0945,  0.2374, -0.2804]]])
└── 'x' --> <FastTreeValue 0x7f27f7582be0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.2 µs ± 61.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f28d187aa30>
├── 'a' --> tensor([[ 0.9911, -0.1099, -2.0049],
│                   [-1.0945,  0.2374, -0.2804],
│                   [ 0.9911, -0.1099, -2.0049],
│                   [-1.0945,  0.2374, -0.2804],
│                   [ 0.9911, -0.1099, -2.0049],
│                   [-1.0945,  0.2374, -0.2804],
│                   [ 0.9911, -0.1099, -2.0049],
│                   [-1.0945,  0.2374, -0.2804],
│                   [ 0.9911, -0.1099, -2.0049],
│                   [-1.0945,  0.2374, -0.2804],
│                   [ 0.9911, -0.1099, -2.0049],
│                   [-1.0945,  0.2374, -0.2804],
│                   [ 0.9911, -0.1099, -2.0049],
│                   [-1.0945,  0.2374, -0.2804],
│                   [ 0.9911, -0.1099, -2.0049],
│                   [-1.0945,  0.2374, -0.2804]])
└── 'x' --> <FastTreeValue 0x7f28d187a7f0>
    └── 'c' --> tensor([[ 0.3893, -2.6018,  0.6255, -1.8468],
                        [-0.0036, -3.4258,  0.8249,  0.3328],
                 

In [23]:
%timeit t_cat(trees)

31 µs ± 75.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.2 µs ± 463 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.9911, -0.1099, -2.0049],
                [-1.0945,  0.2374, -0.2804]],
       
               [[ 0.9911, -0.1099, -2.0049],
                [-1.0945,  0.2374, -0.2804]],
       
               [[ 0.9911, -0.1099, -2.0049],
                [-1.0945,  0.2374, -0.2804]],
       
               [[ 0.9911, -0.1099, -2.0049],
                [-1.0945,  0.2374, -0.2804]],
       
               [[ 0.9911, -0.1099, -2.0049],
                [-1.0945,  0.2374, -0.2804]],
       
               [[ 0.9911, -0.1099, -2.0049],
                [-1.0945,  0.2374, -0.2804]],
       
               [[ 0.9911, -0.1099, -2.0049],
                [-1.0945,  0.2374, -0.2804]],
       
               [[ 0.9911, -0.1099, -2.0049],
                [-1.0945,  0.2374, -0.2804]]]),
    x: Batch(
           c: tensor([[[ 0.3893, -2.6018,  0.6255, -1.8468],
                       [-0.0036, -3.4258,  0.8249,  0.3328],
                       [ 2.0975, -2.0692, -0.6502, -1.6187]],
         

In [26]:
%timeit Batch.stack(batches)

79.6 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.9911, -0.1099, -2.0049],
               [-1.0945,  0.2374, -0.2804],
               [ 0.9911, -0.1099, -2.0049],
               [-1.0945,  0.2374, -0.2804],
               [ 0.9911, -0.1099, -2.0049],
               [-1.0945,  0.2374, -0.2804],
               [ 0.9911, -0.1099, -2.0049],
               [-1.0945,  0.2374, -0.2804],
               [ 0.9911, -0.1099, -2.0049],
               [-1.0945,  0.2374, -0.2804],
               [ 0.9911, -0.1099, -2.0049],
               [-1.0945,  0.2374, -0.2804],
               [ 0.9911, -0.1099, -2.0049],
               [-1.0945,  0.2374, -0.2804],
               [ 0.9911, -0.1099, -2.0049],
               [-1.0945,  0.2374, -0.2804]]),
    x: Batch(
           c: tensor([[ 0.3893, -2.6018,  0.6255, -1.8468],
                      [-0.0036, -3.4258,  0.8249,  0.3328],
                      [ 2.0975, -2.0692, -0.6502, -1.6187],
                      [ 0.3893, -2.6018,  0.6255, -1.8468],
                      [-0.0036, -

In [28]:
%timeit Batch.cat(batches)

148 µs ± 612 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

343 µs ± 2.76 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
